 I am using loan Data ( Dummy Data ) in this project. I am using Numy libirary in Python to clean and preprocess it so that its more applicable for further invistigation using machine learning models.
    
Data used in this project : https://drive.google.com/file/d/12uCRK5bLa70rbSYcogaGypbsyz7u5DMB/view?usp=sharing
<hr style="border:5px solid #108999"> </hr>

# importing Data

In [ ]:
import numpy as np

In [ ]:
np.set_printoptions(suppress=True)

The first step is to insert the CSV file and specify the delimiter ';'

In [ ]:
loan_data=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv', delimiter=';')
loan_data

here we can see that there are alot of nan values which indicates that there are alot of non numeric values 
so the first step is to know the columns that entirely hold strings and we can do that using 'nanmean()' function

In [ ]:
temp=np.nanmean(loan_data, axis=0)
temp

using the next two code lines we can separate the numeric and non-numeric columns indixes in different two arrays (string_columns, numeric_columns)

In [ ]:
string_columns=np.argwhere(np.isnan(temp)).squeeze()
string_columns

In [ ]:
numeric_columns=np.argwhere(np.isnan(temp)==False).squeeze()
numeric_columns

now, we can import the CSV file into four different numpy arrays header numeric, header string, data numerica and data string


In [ ]:
loan_data_numeric=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv',
                                delimiter=';',
                               usecols=numeric_columns,
                               skip_header=1)
loan_data_numeric

In [ ]:
loan_data_string=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv',
                                delimiter=';',
                               usecols=string_columns,
                               skip_header=1,
                               dtype=str)
loan_data_string

In [ ]:
loan_head_string=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv',
                                delimiter=';',
                               usecols=string_columns,
                               skip_footer=loan_data_string.shape[0],
                               dtype=str)
loan_head_string

In [ ]:
loan_head_numeric=np.genfromtxt('D:\Data Processing with Numpy\\loan-data.csv',
                                delimiter=';',
                               usecols=numeric_columns,
                               skip_footer=loan_data_string.shape[0],
                               dtype=str)
loan_head_numeric


<hr style="border:5px solid #108999"> </hr> 

# String data Processing

After separating our data without the headers into 2 arrays we will start working on them one at a time let's start with the string data which consists of columns: issue_d, loan_status, term, grade, sub_grade, verification_status, url and addr_state

In [ ]:
np.unique(loan_data_string[:,0])

### Issue_date
we checked the unique values of the first column in the string data array. as it's obvious, we can see that dates consists of days and months only also, all loans dates are in the same day but different month. so there is no use of mintioning the day
and we can also replace the month name with its numerical value (1-12) and the missing values with 0

In [ ]:
loan_data_string[:,0]=np.chararray.strip(loan_data_string[:,0],'-15')
loan_data_string[:,0]

In [ ]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [ ]:
for i in range(months.shape[0]):
    loan_data_string[:,0]=np.where(loan_data_string[:,0]==months[i],
                                  i,
                                  loan_data_string[:,0])
np.unique(loan_data_string[:,0])

### loan_status
we checked the unique values of the second column 'Loan_statuts' in the string data array. according to the business model we should catigorize these values into two groups (good '1', bad '0')

ps: the Empty values ' ' means there were missing values in the CSV file so we assign them according to the worest case scenario

In [ ]:
np.unique(loan_data_string[:,1])

In [ ]:
status_bad = np.array(['','Charged Off','Default','Late (31-120 days)'])

In [ ]:
loan_data_string[:,1]= np.where(np.isin(loan_data_string[:,1], status_bad),0,1)
np.unique(loan_data_string[:,1])

### Term
lets move to the third column 'term' which contains three unique values ('', '36 months','60 months').
we will replace '36 months' with '36' , '60 months' with '60' and ' ' with '36' as it's the worest case scenario

In [ ]:
np.unique(loan_data_string[:,2])

In [ ]:
loan_data_string[:,2]=np.chararray.strip(loan_data_string[:,2], ' months')
np.unique(loan_data_string[:,2])

In [ ]:
loan_data_string[:,2]=np.where(loan_data_string[:,2]=='',
                              60,
                              loan_data_string[:,2])
np.unique(loan_data_string[:,2])

### grade and sub_grade
in the next two columns (grade column, Sub-grade column),<br>
the unique values for the grade column are (' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G').<br>
and for the sub grade are the same like the grade but a numeric value from 1-5 is added after each grade.<br> so we can<br>
1- Replace the missing values of sub-grade with the the least level of grade if found for example if the grade is 'B' so the missing sub grade is 'B5'. <br>2- delete the grade column as all its info is duplicated in sub-grade.<br>3- replace the missing subgrades with a newly added value 'H1'<br>4- replace every level of sub-grade with a numeric value

In [ ]:
np.unique(loan_data_string[:,4], return_counts=True)

In [ ]:
# 1- Replace the missing values of sub-grade with the the least level of grade if found 
#    for example if the grade is 'B' so the missing sub grade is 'B5'.

for i in range(loan_data_string.shape[0]):
    loan_data_string[i,4]=np.where((loan_data_string[i,4]=='')&(loan_data_string[i,3]!=''),
                                  loan_data_string[i,3]+'5',
                                  loan_data_string[i,4])
    
np.unique(loan_data_string[:,4], return_counts=True)

In [ ]:
# 2- delete the grade column as all its info is duplicated in sub-grade.

loan_data_string=np.delete(loan_data_string, 3, axis=1)
loan_data_string[:,3]
loan_head_string=np.delete(loan_head_string, 3)
loan_head_string[3]

In [ ]:
# 3- replace the missing subgrades with a newly added value 'H1'

loan_data_string[:,3]=np.where(loan_data_string[:,3]=='',
                              'H1',
                              loan_data_string[:,3])

np.unique(loan_data_string[:,3], return_counts=True)

In [ ]:
# 4- replace every level of sub-grade with a numeric value

keys= list(np.unique(loan_data_string[:,3]))
values= list(range(1,np.unique(loan_data_string[:,3]).shape[0]+1))
temp_dict=dict(zip(keys, values))
temp_dict

In [ ]:
for i in temp_dict.keys():
    loan_data_string[:,3]=np.where(loan_data_string[:,3]==i,
                                  temp_dict[i],
                                  loan_data_string[:,3])
np.unique(loan_data_string[:,3])

### Verification Status

for column 'verification_status', the unique values in this column are (' ',Not Verified, Sourece Verified, Verified)
so we can process these data in the following steps:<br>1- replace values (' ',Not Verified) with 0<br>2- replace (Sourece Verified, Verified) with 1

In [ ]:
np.unique(loan_data_string[:,4])

In [ ]:
loan_data_string[:,4]=np.where((loan_data_string[:,4]=='')|(loan_data_string[:,4]=='Not Verified'),
                              0,1)
np.unique(loan_data_string[:,4])

### URL
for the column 'URL' we can see that all the values like this 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226' with except change in the id part at the end of the URL also we can see that this id at the end is the same as the id column in the numeric data<br>Then there is no need for this column as it doesn't add value 

In [ ]:
loan_data_string[:5,5]

In [ ]:
loan_data_numeric[:5,0]

In [ ]:
# the first 5 rows from URL column 
loan_data_string=np.delete(loan_data_string, 5, axis=1)
loan_head_string=np.delete(loan_head_string, 5)
print(loan_head_string)
print(loan_data_string)

### State Address
for column State Address I renamed to be more understandable. The values in that columns is the abbreviations of all the states in United States plus some missing values so<br>1- the first thing to do is to replace the missing values with 0<br> when examining the number of records for each state I found out that every state separately does not give any insights as the the number of records are very low so <br>2- it would be better to divide them into 4 catigories (states_west, states_south, states_midwest, states_east)

In [ ]:
loan_head_string[-1]='state_address'

In [ ]:
# 1- the first thing to do is to replace the missing values with 0
loan_data_string[:,5] = np.where(loan_data_string[:,5] == '', 
                                  0, 
                                  loan_data_string[:,5])

In [ ]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [ ]:
# 2- it would be better to divide them into 4 catigories (states_west, states_south, states_midwest, states_east)

loan_data_string[:,-1] = np.where(np.isin(loan_data_string[:,-1], states_west), 1, loan_data_string[:,-1])
loan_data_string[:,-1] = np.where(np.isin(loan_data_string[:,-1], states_south), 2, loan_data_string[:,-1])
loan_data_string[:,-1] = np.where(np.isin(loan_data_string[:,-1], states_midwest), 3, loan_data_string[:,-1])
loan_data_string[:,-1] = np.where(np.isin(loan_data_string[:,-1], states_east), 4, loan_data_string[:,-1])

np.unique(loan_data_string[:,-1])

### checking

In [ ]:
loan_data_string

In [ ]:
counter=0
for i in loan_data_string.flatten():
    if i=='':
        counter+=1
counter

In [ ]:
loan_data_string=loan_data_string.astype(int)
loan_data_string

<hr style="border:5px solid #108999"> </hr> 


# Numeric data processing

Now its time to process the numeric Data which is the cloumns ['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment',
'total_pymnt']<br>through these following steps we can alter these columns so that we get an array with no missing values and ready for processing <br>1- replace missing values with vallues of the worest case scenario
   

In [ ]:
loan_head_numeric

### loan ID

from this line we can see that the first column 'ID' doesnt have any missing values as expected.

In [ ]:
np.isnan(loan_data_numeric[:,0]).sum()

### Loan Amount, Installments, Total Payments 
PS: the worest case scenario is to replace the messing values for Loaned Amount, Interest Rate, Total Payment and Installment with the maximum of these columns and the minimum of funded amount

In [ ]:
max=np.nanmax(loan_data_numeric[:,(1,3,4,5)], axis=0)
np.nanmean(loan_data_numeric[:,[1,3,4,5]],axis=0)

In [ ]:
counter=0
for i in [1,3,4,5]:
    loan_data_numeric[:,i]= np.where(np.isnan(loan_data_numeric[:,i]),
                               max[counter],
                               loan_data_numeric[:,i])
    counter+=1
np.nanmean(loan_data_numeric[:,[1,3,4,5]],axis=0)

In [ ]:
# to display the interest rate as a decimal value 
loan_data_numeric[:,3]=loan_data_numeric[:,3]/100
loan_data_numeric[:,3]

### Funded Rate

In [ ]:
np.isnan(loan_data_numeric[:,2]).sum()

In [ ]:
min=np.nanmin(loan_data_numeric[:,2])
np.nanmean(loan_data_numeric[:,2])

In [ ]:
loan_data_numeric[:,2]= np.where(np.isnan(loan_data_numeric[:,2]),
                               min,
                               loan_data_numeric[:,2])
np.nanmean(loan_data_numeric[:,2])

### The Exchange Rate
here I used another dataset for the exchange (EUR - USD) it consists of 12 rows for 12 months of the year 2015 in which this loan data is collected.<br>so we want to create another column that contains the exchange rates for each loan and attach it to loan_data_neumeric array

In [ ]:
EUR_USD = np.genfromtxt("D:\\Data Processing with Numpy\\EUR-USD.csv", delimiter = ',', autostrip = True, skip_header = 1, usecols = 3)
EUR_USD

In [ ]:
loan_data_string[0,0]

In [ ]:

exchange_rate = loan_data_string[:,0]

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    

exchange_rate = np.where(exchange_rate == 0,
                         np.mean(EUR_USD),
                         exchange_rate)

exchange_rate

In [ ]:
exchange_rate.shape

In [ ]:
exchange_rate=np.reshape(exchange_rate,(10000,1))


In [ ]:
loan_data_numeric=np.hstack((loan_data_numeric,exchange_rate))
loan_data_numeric[:,-1]

In [ ]:
loan_head_numeric=np.hstack((loan_head_numeric, np.array(['Exchange_Rate'])))

In [ ]:
loan_head_numeric

### from USD to EUR
here we want to change the numerical values in columns (Loan amount, Installments, total payments, fund Rates) from USD to EUR using the newly attached column (exchange rate )

In [ ]:
counter=0
EUR_amount=np.zeros((loan_data_numeric.shape[0],4))

for i in [1,2,4,5]:
    EUR_amount[:,counter]=loan_data_numeric[:,i]/loan_data_numeric[:,6]
    counter+=1

EUR_amount


In [ ]:
loan_data_numeric=np.hstack((loan_data_numeric, EUR_amount))
loan_data_numeric.shape

In [ ]:
# adding EUR columns name to the header

loan_head_numeric=np.hstack((loan_head_numeric,np.array([i+'_EUR' for i in loan_head_numeric[[1,2,4,5]]])))
loan_head_numeric

In [ ]:
# adding _USD to the columns name to specify their currency

for i in [1,2,4,5]:
    loan_head_numeric[i]+='_USD'
loan_head_numeric

In [ ]:
# rearranging the columns
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]
loan_data_numeric=loan_data_numeric[:,columns_index_order]
loan_head_numeric=loan_head_numeric[columns_index_order]
loan_head_numeric

<hr style="border:5px solid #108999"> </hr> 

# Create the complete Dataset

what I will do here :<br>1- concatenate the headers (string, numeric)<br>2-concatenate the data columns<br>3- sort the records ascending depending on the id

In [ ]:
# 1- concatenate the headers (string, numeric)
loan_header=np.hstack((loan_head_numeric, loan_head_string))
loan_header

In [ ]:
# 2-concatenate the data columns

loan_data=np.hstack((loan_data_numeric, loan_data_string))
loan_data

In [ ]:
# 3- sort the records ascending depending on the id

loan_data=loan_data[np.argsort(loan_data[:,0]),:]
loan_data

In [ ]:
loan_header=loan_header.reshape((1,17))
loan_header.shape

In [ ]:
loan_data_Final=np.vstack((loan_header, loan_data))
loan_data_Final

In [ ]:
np.savetxt("D:\\Data Processing with Numpy\\loan-data-preprocessed.csv", 
           loan_data_Final, 
           fmt = '%s',
           delimiter = ',')